In [ ]:
import psycopg2 as pg
import pandas as pd
import os
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

conn = pg.connect(os.getenv('AI_DB_URL'))
open_ai_key = os.environ["OPENAI_API_KEY"]

# INPUTS
# 0: tokenize | 1: detokenize
action = 0
# choose from dictionary models directly below
model_num = 26 
token_string = 'Timescale is Postgres made Powerful'
detoken_array = [1820,25977,46840,23874,389,264,2579,58466]

# from running config_openai.ipynb
models = {
            0: 'gpt-4o-2024-05-13',
            1: 'gpt-4o',
            2: 'gpt-4-turbo-2024-04-09',
            3: 'gpt-4-turbo',
            4: 'gpt-3.5-turbo-0125',
            5: 'gpt-4-turbo-preview',
            6: 'gpt-4-0125-preview',
            7: 'text-embedding-3-large',
            8: 'text-embedding-3-small',
            9: 'tts-1-hd-1106',
            10: 'tts-1-1106',
            11: 'tts-1-hd',
            12: 'gpt-3.5-turbo-1106',
            13: 'gpt-4-1106-preview',
            14: 'dall-e-2',
            15: 'dall-e-3',
            16: 'gpt-3.5-turbo-instruct-0914',
            17: 'gpt-3.5-turbo-instruct',
            18: 'babbage-002',
            19: 'davinci-002',
            20: 'gpt-4',
            21: 'gpt-4-0613',
            22: 'gpt-3.5-turbo-16k',
            23: 'tts-1',
            24: 'gpt-3.5-turbo',
            25: 'whisper-1',
            26: 'text-embedding-ada-002'
}

class OpenaiAction:
    def __init__(self, action, model, input_value):
        self.action = action
        self.model = model 
        self.input_value = input_value
        
    def call_openai(self):
        try:
            cur = conn.cursor()
            if self.action == 0:
                query = '''
                    SELECT openai_tokenize ( %s, %s);
                '''                    
                cur.execute(query, (self.model, self.input_value[0]))
                data_list = cur.fetchall()[0][0]
                return {'data': data_list, 'tokens': len(data_list)}
            elif self.action == 1:
                query = '''
                    SELECT openai_detokenize(%s, %s)
                '''
                cur.execute(query, (self.model, self.input_value[1]))
                return {'data': cur.fetchall()[0][0]}
            else:
                print('only action values are 0 & 1. Currently assigned as: {}'.format(self.action))
        except Exception as e:
            print('Error in call_openai: {}'.format(e))   

CallApi = OpenaiAction(action=action, model=models[model_num], input_value=[token_string, detoken_array])
    
CallApi.call_openai()